# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_file = "cities.csv"
data_file_df = pd.read_csv(data_file)
data_file_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Arraial do Cabo,0,BR,1612391084,78,-22.9661,-42.0278,82.40,21.85
1,1,Bengkulu,100,ID,1612390583,86,-3.8004,102.2655,75.74,5.35
2,2,Dikson,100,RU,1612390562,91,73.5069,80.5464,-3.53,15.17
3,3,Atuona,1,PF,1612390576,72,-9.8000,-139.0333,78.01,10.78
4,4,Albany,100,US,1612390325,66,42.6001,-73.9662,30.00,11.99


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
humidity = data_file_df['Humidity']
locations = data_file_df[['Lat', 'Lng']]
locations
fig=gmaps.figure(center=(50.0, -35.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                  dissipating=False, max_intensity=200,
                                 point_radius=3)
heat_layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
reduced_cities_df = data_file_df.loc[(data_file_df["Wind Speed"] < 10) & (data_file_df["Cloudiness"] == 0) & \
                                   (data_file_df["Max Temp"] > 70) & (data_file_df["Max Temp"] < 80)].dropna()

reduced_cities_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
23,23,Azare,0,NG,1612391087,20,11.6765,10.1948,71.04,9.93
26,26,Belmonte,0,BR,1612391087,78,-15.8631,-38.8828,78.80,6.91
41,41,Eilat,0,IL,1612391089,78,29.5581,34.9482,71.60,4.61
69,69,Saint-Philippe,0,RE,1612390580,78,-21.3585,55.7679,77.00,3.44
102,102,Saint-Joseph,0,RE,1612391097,78,-21.3667,55.6167,77.00,3.44
150,150,Saint-Pierre,0,RE,1612391102,78,-21.3393,55.4781,77.00,3.44
170,170,Maceió,0,BR,1612391103,83,-9.6658,-35.7353,77.00,6.91
179,179,Koungou,0,YT,1612391106,89,-12.7336,45.2042,78.80,6.91
231,231,Saint-Louis,0,SN,1612391114,29,16.3333,-15.0000,77.00,2.30
316,316,Doumé,0,CM,1612391128,56,4.2333,13.4500,74.97,3.09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = reduced_cities_df.loc[:,["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
23,Azare,NG,11.6765,10.1948,
26,Belmonte,BR,-15.8631,-38.8828,
41,Eilat,IL,29.5581,34.9482,
69,Saint-Philippe,RE,-21.3585,55.7679,
102,Saint-Joseph,RE,-21.3667,55.6167,


In [6]:
import json

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("No hotel")
        
    print("------------")

Retrieving Results for Index 23: Azare.
Closest hotel in Azare is Hotel.
------------
Retrieving Results for Index 26: Belmonte.
Closest hotel in Belmonte is Pousada do Diogo.
------------
Retrieving Results for Index 41: Eilat.
Closest hotel in Eilat is Queen of Sheba Eilat.
------------
Retrieving Results for Index 69: Saint-Philippe.
Closest hotel in Saint-Philippe is Pinpin d'Amour.
------------
Retrieving Results for Index 102: Saint-Joseph.
Closest hotel in Saint-Joseph is "Plantation Bed and Breakfast.
------------
Retrieving Results for Index 150: Saint-Pierre.
Closest hotel in Saint-Pierre is LODGE PALMAE.
------------
Retrieving Results for Index 170: Maceió.
Closest hotel in Maceió is Jatiúca Hotel & Resort.
------------
Retrieving Results for Index 179: Koungou.
Closest hotel in Koungou is Le Lodge de l''Avocatier.
------------
Retrieving Results for Index 231: Saint-Louis.
No hotel
------------
Retrieving Results for Index 316: Doumé.
Closest hotel in Doumé is Hotel Univer

In [7]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
23,Azare,NG,11.6765,10.1948,Hotel
26,Belmonte,BR,-15.8631,-38.8828,Pousada do Diogo
41,Eilat,IL,29.5581,34.9482,Queen of Sheba Eilat
69,Saint-Philippe,RE,-21.3585,55.7679,Pinpin d'Amour
102,Saint-Joseph,RE,-21.3667,55.6167,"""Plantation Bed and Breakfast"


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))